# ELMO!!!

In [2]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
!pip install razdel

In [4]:
!pip install razdel
!pip install deeppavlov
!pip install tensorflow==1.15
# !pip install transformers


  Using cached https://files.pythonhosted.org/packages/92/2b/e3af15221da9ff323521565fa3324b0d7c7c5b1d7a8ca66984c8d59cb0ce/tensorflow-1.15.0-cp37-cp37m-manylinux2010_x86_64.whl


In [ ]:
ftom_b = 22
to_b = 100

import psutil
import humanize
import os
import GPUtil as GPU
def printm(text):
 process = psutil.Process(os.getpid())
 print("{}_Gen RAM Free: ".format(text) + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
#  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm("S")
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/Shared drives/hse_BERT/hse_Af_Tr_BERT')

folder = "elmo-DeepPavlov-liter"
# name = "all_cyberleninka_polit2"
name = "bin/all_cyberleninka_liter2"

if not os.path.isdir(os.path.join("embeddings", folder)):
    os.mkdir(os.path.join("embeddings", folder))
if not os.path.isdir(os.path.join("embeddings", folder, "bin")):
    os.mkdir(os.path.join("embeddings", folder, "bin"))
if not os.path.isdir(os.path.join("embeddings", folder, "ready")):
    os.mkdir(os.path.join("embeddings", folder, "ready"))
if not os.path.isdir(os.path.join("embeddings", folder, "tests")):
    os.mkdir(os.path.join("embeddings", folder, "tests"))

import torch
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import gc
import json
import time
import numpy as np
import pandas as pd
import pickle as pc
# from transformers import BertTokenizer, BertModel

import matplotlib.pyplot as plt
% matplotlib inline

from scipy.spatial.distance import cosine
from razdel import sentenize

import tensorflow as tf
import tensorflow_hub as hub

def my_dump(file, name):
    with open("./" + name + ".pic", 'wb') as f:
        pc.dump(file, f)

def my_load(name):
    with open("./" + name + ".pic", 'rb') as f:
        return pc.load(f)

elmo = hub.Module("http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz",trainable=True)
# elmo = hub.Module("http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-wiki_600k_steps.tar.gz",trainable=True)

def get_batch_number(from_i = 100, to_i = 1000):
    
    files_r = os.listdir("./embeddings/{}/bin".format(folder))
    cut_files_r = []
    for i in files_r:
        if i.split('.')[-1] == 'pic':
            cut_files_r.append(int(i.split('.')[0].split('_')[1].split(" ")[0]))
    diff = set(range(from_i, to_i)) - set(cut_files_r)
    return sorted(list(diff))

sent1 = my_load(name)

def get_text_from_ST(ss):
    return ss.text
sent = list(map(get_text_from_ST, sent1))
len(sent)

sent2 = sent[:1000000]
del sent
del sent1

sub = re.compile(r'[^\w\s]')
sub2 = re.compile(r'[\d]')
# text = "Тюмень стала самым «счастливым» городом России по уровню довольных своим городом людей."

def get_tokens(text):
    # printm("gt1")
    opt = sub.sub("", text.lower())    
    opt = sub2.sub("", opt)
    # printm("gt2")  
    return word_tokenize(opt, language="russian")

def elmo_vectors(text):
    # printm("ev1")
    tokens = get_tokens(text)
    # printm("ev2")
    with tf.Session() as sess:
        embeddings = elmo(tokens, signature="default", as_dict=True)["elmo"]
        # printm("ev3")
    # with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        # return average of ELMo features
        av_elmo = sess.run(tf.reduce_mean(embeddings,1))
        # tf.contrib.keras.backend.clear_session()
    # printm("ev4")
    del embeddings
    
    # printm("ev5")
    return tokens, av_elmo

# %%time
batch = 10
start_time = time.time()

gc.enable()



print("senteces from {} to {}".format(ftom_b * batch, to_b * batch))
l = get_batch_number()
while len(l) > 0:
    j = l[11]
    pr_tr = {}
    temp_time = time.time()
    for i in range(j * batch, (j + 1) * batch):
        printm("st1")
        tt, tvs = elmo_vectors(sent2[i])
        for i in range(len(tt)):
            if tt[i].isalpha() and len(tt[i]) > 2 :
                if tt[i] in pr_tr:
                    pr_tr[tt[i]].append(tvs[i])
                else:
                    pr_tr[tt[i]] = [tvs[i]]

    with open("./embeddings/{}/bin/{}_{:05d}.pic".format(folder, batch, j), 'wb') as f:
        pc.dump(pr_tr, f)
    l = get_batch_number()

    del pr_tr

    printm("gc1")
    print(gc.collect())
    printm("gc2")
    t = time.time()
    print("\n done {:05d} --- time: {}\t--- total: {} \n".format(j, t - temp_time, t - start_time))

# for i in pr_tr:
#     print(i, len(pr_tr[i]))

S_Gen RAM Free: 12.8 GB  | Proc size: 117.5 MB
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


senteces from 220 to 1000
st1_Gen RAM Free: 11.4 GB  | Proc size: 5.1 GB
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


st1_Gen RAM Free: 11.2 GB  | Proc size: 6.1 GB
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

st1_Gen RAM Free: 3.9 GB  | Proc size: 12.0 GB
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


st1_Gen RAM Free: 2.2 GB  | Proc size: 12.0 GB
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


st1_Gen RAM Free: 1.6 GB  | Proc size: 12.2 GB


# Check RAM

In [2]:
def printm(text):
 process = psutil.Process(os.getpid())
 print("{}_Gen RAM Free: ".format(text) + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
#  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm("S")

S_Gen RAM Free: 12.8 GB  | Proc size: 118.9 MB


# Подготовка среды

In [1]:
# !pip install transformers
!pip install razdel

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
os.chdir('/content/drive/Shared drives/hse_BERT/hse_Af_Tr_BERT')

In [4]:
folder = "elmo-DeepPavlov-liter"

if not os.path.isdir(os.path.join("embeddings", folder)):
    os.mkdir(os.path.join("embeddings", folder))
if not os.path.isdir(os.path.join("embeddings", folder, "bin")):
    os.mkdir(os.path.join("embeddings", folder, "bin"))
if not os.path.isdir(os.path.join("embeddings", folder, "ready")):
    os.mkdir(os.path.join("embeddings", folder, "ready"))
if not os.path.isdir(os.path.join("embeddings", folder, "tests")):
    os.mkdir(os.path.join("embeddings", folder, "tests"))

In [5]:
import torch
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import gc
import json
import time
import numpy as np
import pandas as pd
import pickle as pc
# from transformers import BertTokenizer, BertModel

import matplotlib.pyplot as plt
% matplotlib inline

from scipy.spatial.distance import cosine
from razdel import sentenize

import tensorflow as tf
import tensorflow_hub as hub

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
def my_dump(file, name):
    with open("./" + name + ".pic", 'wb') as f:
        pc.dump(file, f)

def my_load(name):
    with open("./" + name + ".pic", 'rb') as f:
        return pc.load(f)

In [7]:
# from deeppavlov.models.embedders.elmo_embedder import ELMoEmbedder
# elmo = ELMoEmbedder("http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz")

In [8]:
elmo = hub.Module("http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz",trainable=True)
# elmo = hub.Module("http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-wiki_600k_steps.tar.gz",trainable=True)


In [10]:
# Load pre-trained model tokenizer (vocabulary)
# 'DeepPavlov/rubert-base-cased-sentence'
# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [ ]:
# model = BertModel.from_pretrained('bert-base-multilingual-cased',
#                                   output_hidden_states = True, # Whether the model returns all hidden-states.
#                                   )
# model.eval()

#text processing

## generate (split) sentences

In [ ]:
%%time
# name = "all_cyberleninka_polit2"
name = "all_cyberleninka_liter2"
with open("./texts/{}/{}.txt".format(name, name), 'r') as f:
    data = f.read()
sent2 = list(sentenize(data))
my_dump(sent2, name)

CPU times: user 6min 1s, sys: 12.6 s, total: 6min 14s
Wall time: 6min 48s


#getting embeddings

## read sentences

In [10]:
%%time
# name = "all_cyberleninka_polit2"
name = "bin/all_cyberleninka_liter2"
sent1 = my_load(name)

CPU times: user 20.2 s, sys: 4.85 s, total: 25.1 s
Wall time: 32 s


In [11]:
def get_text_from_ST(ss):
    return ss.text
sent = list(map(get_text_from_ST, sent1))
len(sent)

6287032

In [12]:
sent2 = sent[:1000000]
del sent
del sent1

##process

In [13]:
import gc

In [14]:
sub = re.compile(r'[^\w\s]')
sub2 = re.compile(r'[\d]')
# text = "Тюмень стала самым «счастливым» городом России по уровню довольных своим городом людей."

def get_tokens(text):
    # printm("gt1")
    opt = sub.sub("", text.lower())    
    opt = sub2.sub("", opt)
    # printm("gt2")  
    return word_tokenize(opt, language="russian")

def elmo_vectors(text):
    # printm("ev1")
    tokens = get_tokens(text)
    # printm("ev2")
    with tf.Session() as sess:
        embeddings = elmo(tokens, signature="default", as_dict=True)["elmo"]
        # printm("ev3")
    # with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        # return average of ELMo features
        av_elmo = sess.run(tf.reduce_mean(embeddings,1))
        # tf.contrib.keras.backend.clear_session()
    # printm("ev4")
    del embeddings
    
    # printm("ev5")
    return tokens, av_elmo


# def get_embeddings_elmo(text):
#     tokens = get_tokens(text)
#     if len(tokens) > 80:
#         return [],[]
#     sess = tf.Session()
#     sess.run(tf.global_variables_initializer())
#     embeddings = elmo(inputs={"tokens": [tokens],"sequence_len": [len(tokens)]},signature="tokens",as_dict=True)["elmo"]
#     token_vecs_sum = sess.run(embeddings)
#     sess.close()
#     return tokens, token_vecs_sum[0]

# def get_embeddings_elmo3(text):
#     tokens = get_tokens(text)
#     if len(tokens) > 90:
#         return [],[]
#     token_vecs_sum = elmo(tokens)
#     return tokens, token_vecs_sum[0]

# def get_embeddings_elmo2(texts):
#     tokens = list(map(get_tokens, texts))
#     lengths = list(map(len, tokens))
#     # if len(tokens) > 80:
#     #     return [],[]
#     sess = tf.Session()
#     sess.run(tf.global_variables_initializer())
#     embeddings = elmo(inputs={"tokens": tokens,"sequence_len": lengths},signature="tokens",as_dict=True)["elmo"]
#     token_vecs_sum = sess.run(embeddings)
#     sess.close()
#     return tokens, token_vecs_sum


# # token_vecs = hidden_states[-2][0]
# # sentence_embedding = torch.mean(token_vecs, dim=0)

In [15]:
# get_embeddings_elmo(sent[0])

In [7]:
def get_batch_number(from_i = 0, to_i = 100):
    
    files_r = os.listdir("./embeddings/{}/bin".format(folder))
    cut_files_r = []
    for i in files_r:
        if i.split('.')[-1] == 'pic':
            cut_files_r.append(int(i.split('.')[0].split('_')[1].split(" ")[0]))
    diff = set(range(from_i, to_i)) - set(cut_files_r)
    return sorted(list(diff))

In [ ]:
# %%time
batch = 10
start_time = time.time()

gc.enable()

ftom_b = 12
to_b = 20

print("senteces from {} to {}".format(ftom_b * batch, to_b * batch))

for j in range(ftom_b, to_b):
    pr_tr = {}
    temp_time = time.time()
    for i in range(j * batch, (j + 1) * batch):
        printm("st1")
        tt, tvs = elmo_vectors(sent2[i])
        for i in range(len(tt)):
            if tt[i].isalpha() and len(tt[i]) > 2 :
                if tt[i] in pr_tr:
                    pr_tr[tt[i]].append(tvs[i])
                else:
                    pr_tr[tt[i]] = [tvs[i]]

    with open("./embeddings/{}/bin/{}_{:05d}.pic".format(folder, batch, j), 'wb') as f:
        pc.dump(pr_tr, f)

    del pr_tr

    printm("gc1")
    print(gc.collect())
    printm("gc2")
    t = time.time()
    print("done {:05d} --- time: {}\t--- total: {}".format(j, t - temp_time, t - start_time))

# for i in pr_tr:
#     print(i, len(pr_tr[i]))

senteces from 120 to 200
st1_Gen RAM Free: 11.4 GB  | Proc size: 5.1 GB
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


st1_Gen RAM Free: 11.2 GB  | Proc size: 6.2 GB
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


st1_Gen RAM Free: 11.2 GB  | Proc size: 6.2 GB
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

st1_Gen RAM Free: 2.6 GB  | Proc size: 12.0 GB
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


st1_Gen RAM Free: 2.1 GB  | Proc size: 12.2 GB
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


st1_Gen RAM Free: 1.6 GB  | Proc size: 12.2 GB


In [28]:
printm("gc1")
gc.collect()
printm("gc2")

gc1_Gen RAM Free: 4.2 GB  | Proc size: 11.8 GB
gc2_Gen RAM Free: 4.2 GB  | Proc size: 11.8 GB


In [31]:
gc.garbage

[]

In [17]:
# batch = 1
# texts = sent2[j * batch: (j + 1) * batch]

In [ ]:
# tokens = list(map(get_tokens, texts))
# lengths = list(map(len, tokens))
# if len(tokens) > 80:


In [ ]:
# tokens

In [ ]:
#     return [],[]
# embeddings = elmo(inputs={"tokens": tokens,"sequence_len": lengths},signature="tokens",as_dict=True)["elmo"]
# token_vecs_sum = sess.run(embeddings)

In [ ]:
# token_vecs_sum

In [ ]:
# # %%time
# batch = 1
# start_time = time.time()

# ftom_b = 0
# to_b = 10

# print("senteces from {} to {}".format(ftom_b * batch, to_b * batch))

# pr_tr = {}
# temp_time = time.time()
# # for i in range(j * batch, (j + 1) * batch):
# j = 0
# tt, tvs = get_embeddings_elmo2(sent2[j * batch: (j + 1) * batch])
# # for i in range(len(tt)):
# #     if tt[i].isalpha() and len(tt[i]) > 2 :
# #         if tt[i] in pr_tr:
# #             pr_tr[tt[i]].append(tvs[i])
# #         else:
# #             pr_tr[tt[i]] = [tvs[i]]

# # for i in pr_tr:
# #     print(i, len(pr_tr[i]))

senteces from 0 to 10
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
# # %%time
# batch = 10
# start_time = time.time()

# ftom_b = 0
# to_b = 10

# print("senteces from {} to {}".format(ftom_b * batch, to_b * batch))

# for j in range(ftom_b, to_b):
#     pr_tr = {}
#     temp_time = time.time()
#     # for i in range(j * batch, (j + 1) * batch):
#     tt, tvs = get_embeddings_elmo2(sent[j * batch: (j + 1) * batch])
#     # for i in range(len(tt)):
#     #     if tt[i].isalpha() and len(tt[i]) > 2 :
#     #         if tt[i] in pr_tr:
#     #             pr_tr[tt[i]].append(tvs[i])
#     #         else:
#     #             pr_tr[tt[i]] = [tvs[i]]

#     with open("./embeddings/{}/bin/{}_{:05d}.pic".format(folder, batch, j), 'wb') as f:
#         pc.dump(pr_tr, f)

#     t = time.time()
#     print("done {:05d} --- time: {}\t--- total: {}".format(j, t - temp_time, t - start_time))

# # for i in pr_tr:
# #     print(i, len(pr_tr[i]))

#aggregation of embeddings M



In [ ]:
%%time
count_t = {}
sem_emb = {}

for name in cut_files:
    with open("./embeddings/{}/bin/{}".format(folder, name), 'rb') as f:
        data = pc.load(f)
    for i in data.keys():
        if i not in count_t:
            count_t[i] = 0
            sem_emb[i] = data[i]
        else:
            count_t[i] += 1
            sem_emb[i] = sem_emb[i] + data[i]

res = {}
for key in count_t:
    res[key] = sem_emb[key] / count_t[key]
    # print(name)

In [ ]:
with open("./embeddings/{}/dict_{}.pic".format(folder, folder), "bw") as f:
    pc.dump(res, f)

#aggregation of embeddings



In [ ]:
# folder = "bert-base-multilingual-cased-liter"

##list of files

In [9]:
files = os.listdir("./embeddings/{}/bin".format(folder))
cut_files = []
for i in files:
    if i.split('.')[-1] == 'pic':
        cut_files.append(i)

##get count

In [10]:
%%time
from_files = {}
all_count = {}

for name in cut_files:
    with open("./embeddings/{}/bin/{}".format(folder, name), 'rb') as f:
        data = pc.load(f)
    for i in data.keys():
        if i not in all_count:
            all_count[i] = len(data[i])
            from_files[i] = [name]
        else:
            all_count[i] += len(data[i])
            from_files[i].append(name)
    # print(name)


CPU times: user 558 ms, sys: 136 ms, total: 694 ms
Wall time: 49.1 s


In [11]:
my_dump(from_files, "embeddings/{}/from_files".format(folder))
my_dump(all_count, "embeddings/{}/all_count".format(folder))

## aggregate

In [12]:
from_files = my_load("embeddings/{}/from_files".format(folder))
all_count = my_load("embeddings/{}/all_count".format(folder))

In [13]:
out = list(zip(all_count.keys(), list(all_count.values())))
out.sort(key = lambda i: i[1], reverse=True)
out[:20]

[('как', 311),
 ('что', 290),
 ('его', 185),
 ('для', 124),
 ('это', 123),
 ('или', 106),
 ('the', 92),
 ('так', 85),
 ('только', 75),
 ('жизни', 65),
 ('более', 62),
 ('текста', 61),
 ('все', 60),
 ('всего', 59),
 ('литературы', 57),
 ('времени', 57),
 ('было', 57),
 ('при', 56),
 ('поэзии', 56),
 ('время', 56)]

In [14]:
keys = list(all_count.keys())
k1 = []
k2 = []
k3 = []
kb = []
l = 0
for i in keys:
    l = len(i)
    if l == 1:
        k1.append(i)
    elif l == 2:
        k2.append(i)
    elif l == 3:
        k3.append(i)
    else:
        kb.append(i)
len(k1),len(k2),len(k3),len(kb)

(0, 0, 343, 14195)

In [15]:
all_words = []
for i in kb:
    if all_count[i] > 0:
        all_words.append(i)
len(all_words)

14195

In [16]:
def check_existing_emb(words):
    d = dict(out)
    fl = True
    for i in words:
        if i not in d:
            print("{} not in dictionary".format(i))
            fl = False
    return fl

def get_all_embeddings(words):
    all_files = []
    embeddings = {}
    for w in words:
        embeddings[w] = []

    for i in words:
        all_files += from_files[i]

    for name in set(all_files):
        with open("./embeddings/{}/bin/{}".format(folder, name), 'rb') as f:
            data = pc.load(f)
        for w in words:
            if w in data:
                embeddings[w] += data[w]
    return embeddings

def sum_embeddings(embeddings):
    res = {}
    res_emb = {}
    for i in embeddings:
        res[i] = np.ndarray([len(embeddings[i]), embeddings[i][0].shape[0]])
        for j in range(len(embeddings[i])):
            res[i][j] = embeddings[i][j]
        res_emb[i] = res[i].mean(0)
    return res_emb

In [17]:
# np.ndarray([4,5]).mean(0)

In [18]:
# batch = 100
# st_time = time.time()

# ftom_b = 0
# to_b = 15

# print("words from {} to {}, max batchs = {}".format(ftom_b * batch, to_b * batch, len(all_words)//batch))

# all_emb = {}
# for b in range(ftom_b, to_b):
#     t_time = time.time()
#     words = all_words[b*batch:(b+1)*batch]
#     emb = get_all_embeddings(words)
#     res_emb = sum_embeddings(emb)
    
#     with open("./embeddings/{}/ready/{}_{:05d}.pic".format(folder,batch, b), 'wb') as f:
#         pc.dump(res_emb, f)

#     print("done: {} --- time: {}\t--- total: {}".format(b,time.time() - t_time, time.time() - st_time))

In [19]:
# emb["творчество"]

In [20]:
def get_batch_number(max_i, from_i = 0):
    
    files_r = os.listdir("./embeddings/{}/ready".format(folder))
    cut_files_r = []
    for i in files_r:
        if i.split('.')[-1] == 'pic':
            cut_files_r.append(int(i.split('.')[0].split('_')[1]))
    diff = set(range(from_i, max_i)) - set(cut_files_r)
    return sorted(list(diff))

def reserve_batch(number):
    print("reserve {}".format(b))
    my_dump("file", "./embeddings/{}/ready/{}_{:05d}_empty".format(folder,batch, b))

batch = 100
from_i = 0
st_time = time.time()
print(" max batchs = {}".format( len(all_words)//batch))

all_emb = {}
l = get_batch_number(len(all_words)//batch + 1, from_i)
while len(l) > 0:
    b = l[0]
    # reserve_batch(b)
    t_time = time.time()
    words = all_words[b*batch:(b+1)*batch]
    emb = get_all_embeddings(words)
    res_emb = sum_embeddings(emb)
    
    with open("./embeddings/{}/ready/{}_{:05d}.pic".format(folder,batch, b), 'wb') as f:
        pc.dump(res_emb, f)

    print("done: {} --- time: {}\t--- total: {}".format(b,time.time() - t_time, time.time() - st_time))
    l = get_batch_number(len(all_words)//batch + 1, from_i)

 max batchs = 141
done: 81 --- time: 0.10198020935058594	--- total: 0.11530756950378418
done: 82 --- time: 0.10973882675170898	--- total: 0.22840189933776855
done: 83 --- time: 0.09584903717041016	--- total: 0.32736730575561523
done: 84 --- time: 0.11682653427124023	--- total: 0.4474313259124756
done: 85 --- time: 0.07462286949157715	--- total: 0.5253696441650391
done: 86 --- time: 0.06529736518859863	--- total: 0.5936195850372314
done: 87 --- time: 0.05226945877075195	--- total: 0.6489474773406982
done: 88 --- time: 0.11127543449401855	--- total: 0.7678494453430176
done: 89 --- time: 0.07420706748962402	--- total: 0.8452813625335693
done: 90 --- time: 0.08195114135742188	--- total: 0.9313497543334961
done: 91 --- time: 0.08380818367004395	--- total: 1.0190439224243164
done: 92 --- time: 0.08863234519958496	--- total: 1.1112377643585205
done: 93 --- time: 0.07299113273620605	--- total: 1.187718391418457
done: 94 --- time: 0.09183907508850098	--- total: 1.2831242084503174
done: 95 --- t

#building dictionary


In [21]:
# folder = "bert-base-multilingual-cased-liter"

In [22]:
files = os.listdir("./embeddings/{}/ready".format(folder))
res = dict()
for i in files:
    with open("./embeddings/{}/ready/{}".format(folder,i), "rb") as f:
        temp = pc.load(f)
        # print(type(temp))
        res.update(temp)

In [23]:
with open("./embeddings/{}/dict_{}.pic".format(folder, folder), "bw") as f:
    pc.dump(res, f)

In [24]:
# with open("./embeddings/{}/dict_{}.json".format(folder, folder), 'w', encoding="utf-8") as f:
#     json.dump(lista_items, f, indent=4)

In [ ]:
res

#Tests

##preparation

In [26]:
# folder = "bert-base-multilingual-cased-liter"

In [27]:
with open("./embeddings/{}/dict_{}.pic".format(folder, folder), "br") as f:
    dct1  = pc.load(f)

# with open("/content/drive/Shareddrives/hse_BERT/hse_Af_Tr_BERT/embeddings/sent1/all_emc.pic", "br") as f:
#     dct1  = pc.load(f)

In [28]:
def get_word(emb, dct, nin = []):
    word = "none"
    cs = -100 
    for key in dct:
        temp_cs = 1 - cosine(emb, dct[key])
        if temp_cs > cs and key not in nin:
            cs = temp_cs
            word = key
    return word, cs

def get_word_m(emb, dct, count = 3):
    res = {}
    for key in dct:
        res[key] = 1 - cosine(emb, dct[key])
    l = list(res.items()) 
    l.sort(key= lambda x:x[1], reverse = True)
    return dict(l[:count])

def add_names(df):
    names = {': capital-common-countries' : "Capitals-common-countries",
 ': capital-world': "Capital-countries",
 ': currency': "Currency",
#  ': city-in-state',
 ': family': "Family",
 ': gram1-adjective-to-adverb': "Adjective adverb",
 ': gram2-opposite': "Opposite",
 ': gram3-comparative': "Comparative",
#  ': gram4-superlative',
#  ': gram5-present-participle',
 ': gram6-nationality-adjective': "Nationality adjective",
#  ': gram7-past-tense',
#  ': gram8-plural',
#  ': gram9-plural-verbs'
    }
    df["type"] = "none"
    for i in names:
        if i in df.index:
            df.loc[i, "type"] = names[i]

    return df

def add_names_rus(df):
    names = {': capital-common-countries' : "Известные столицы - страны",
 ': capital-world': "Столицы - страны",
 ': currency': "Валюта",
#  ': city-in-state',
 ': family': "Семья",
 ': gram1-adjective-to-adverb': "Прилагательное - наречие",
 ': gram2-opposite': "Антонимы",
 ': gram3-comparative': "Сравнение",
#  ': gram4-superlative',
#  ': gram5-present-participle',
 ': gram6-nationality-adjective': "Национальность - прилагательное",
#  ': gram7-past-tense',
#  ': gram8-plural',
#  ': gram9-plural-verbs'
    }
    df["type"] = "none"
    for i in names:
        if i in df.index:
            df.loc[i, "type"] = names[i]

    return df

In [29]:
def save_table(df, name):
    df.to_csv("./embeddings/{}/tests/{}.csv".format(folder, name))

def get_table( name):
    return pd.read_csv("./embeddings/{}/tests/{}.csv".format(folder, name))

In [30]:
# list(dct1.keys())

In [31]:
get_word_m(dct1["россия"], dct1, 10)

{'англия': 0.7826825770543104,
 'венгрия': 0.7545903068007707,
 'испания': 0.7947051717129515,
 'мирена': 0.6787489017454016,
 'оппозиция': 0.6831814782076344,
 'республика': 0.6844801829040114,
 'россию': 0.6994023283547559,
 'россия': 1.0,
 'страна': 0.738829347293551,
 'франция': 0.7796398465720953}

##GATS

###generate test

In [32]:
with open("/content/drive/Shareddrives/hse_BERT/hse_Af_Tr_BERT/tests/questions-words.txt", "r") as f:
    lines = f.read().splitlines()

words = {}
words2 = {}
all_w = []
key = "no_key"
for i in lines:
    m = i.split(" ")
    if i[0] == ":":
        key = i
    else:
        if key not in words:
            words[key] = []
            words2[key] = []
        words[key].append(m)
        all_w = all_w + m
        words2[key].append((m[0], m[1]))
        words2[key].append((m[2], m[3]))
all_w = set(all_w)
for key in words2:
    words2[key] = set(words2[key])

In [33]:
names = [
        #  ': capital-common-countries',
 ': capital-world',
#  ': currency',
#  ': city-in-state',
 ': family',
 ': gram1-adjective-to-adverb',
 ': gram2-opposite',
 ': gram3-comparative',
#  ': gram4-superlative',
#  ': gram5-present-participle',
 ': gram6-nationality-adjective',
#  ': gram7-past-tense',
#  ': gram8-plural',
#  ': gram9-plural-verbs'
 ]

In [34]:
words3 = {}
find_words = []
for key in names:
    df = pd.read_csv("tests/gats/{}.csv".format(key[2:]),sep = ";", encoding = "cp1251")
    d1 = list(zip(df["eng_w1"], df["rus_w1"]))
    d2 = list(zip(df["eng_w2"], df["rus_w2"]))
    words3[key] = dict(d1)
    words3[key].update(dict(d2))
    all_rus = [*df["rus_w1"], *df["rus_w2"]]

In [35]:
def translate_sl(m ,sl):
    m2 = []
    for i in m:
        m2.append(sl[i])
    return m2

In [36]:
words_rus = {}
for key in names:
    words_rus[key] = []
    for i in words[key]:
        words_rus[key].append(translate_sl(i ,words3[key]))

In [37]:
with open(os.path.join("embeddings", folder, "tests", "ready.pic"), "bw") as f:
    pc.dump(words_rus, f)

##get quality


In [38]:
with open(os.path.join("embeddings", folder, "tests", "ready.pic"), "br") as f:
    words_rus = pc.load( f)

In [39]:
def cut_b(text):
    # return tokenizer.tokenize("[CLS] " + text.lower() + " [SEP]")[1]
    return text

###3cosadd

In [40]:
def cos_add(key, dct):
    res_df = pd.DataFrame(columns=["w1","w2","w3", "expected", "prdicted", "cosine", "group"])
    
    for m_l in words_rus[key]:
        m = []
        for i in m_l:
            m.append(cut_b(i))
    for m in words_rus[key]:
        if m[0] in dct and m[1] in dct and m[2] in dct and m[3] in dct:
            pred, prob = get_word(dct[m[0]] - dct[m[1]] + dct[m[3]], dct, nin = [m[0], m[1], m[3]])
            res_df.loc[res_df.shape[0]] = [m[0], m[1], m[3], m[2], pred, prob, key]
            # print(m[2], pred)
    return res_df

In [41]:
n = 3
def cos_add_3(key, dct):
    res_df = pd.DataFrame(columns=["w1","w2","w3", "expected", "prdicted", "cosine", "group"])
    
    for m_l in words_rus[key]:
        m = []
        for i in m_l:
            m.append(cut_b(i))
    for m in words_rus[key]:
        if m[0] in dct and m[1] in dct and m[2] in dct and m[3] in dct:
            pred = get_word_m(dct[m[0]] - dct[m[1]] + dct[m[3]], dct,count = n)
            if m[2] in pred:
                res_df.loc[res_df.shape[0]] = [m[0], m[1], m[3], m[2], m[2], pred[m[2]], key]
            else:
                res_df.loc[res_df.shape[0]] = [m[0], m[1], m[3], m[2], list(pred.keys())[0], pred[list(pred.keys())[0]], key]
            # print(m[2], pred)
    return res_df

In [42]:
for key in words_rus:
    print("start {}".format(key))
    res_df1 = cos_add(key, dct1)
    save_table(res_df1, "3cosadd_res_{}".format(key))
    print("finish {}".format(key))

start : capital-world
finish : capital-world
start : family
finish : family
start : gram1-adjective-to-adverb
finish : gram1-adjective-to-adverb
start : gram2-opposite
finish : gram2-opposite
start : gram3-comparative
finish : gram3-comparative
start : gram6-nationality-adjective
finish : gram6-nationality-adjective


In [43]:
groups = [": capital-world",
": family",
": gram1-adjective-to-adverb",
": gram2-opposite",
": gram3-comparative",
": gram6-nationality-adjective"]

res_df1 = pd.DataFrame()
for key in groups:
    res_df1 = pd.concat([res_df1, get_table("3cosadd_res_{}".format(key))], ignore_index = True )

In [44]:
# get_table("3cosadd_res_{}".format(key))
res_df1

,Unnamed: 0,w1,w2,w3,expected,prdicted,cosine,group
0,0,очевидный,очевидно,полностью,полный,полный,0.697444,: gram1-adjective-to-adverb
1,1,очевидный,очевидно,очевидно,очевидный,красноречивый,0.761310,: gram1-adjective-to-adverb
2,2,очевидный,очевидно,возможно,возможный,возможный,0.773163,: gram1-adjective-to-adverb
3,3,очевидный,очевидно,тихо,тихий,тихий,0.722512,: gram1-adjective-to-adverb
4,4,очевидный,очевидно,редко,редкий,редкий,0.685816,: gram1-adjective-to-adverb
...,...,...,...,...,...,...,...,...
94,6,россия,русский,английский,англия,англия,0.744028,: gram6-nationality-adjective
95,7,россия,русский,французский,франция,франция,0.734582,: gram6-nationality-adjective
96,8,испания,испанский,английский,англия,англия,0.833603,: gram6-nationality-adjective
97,9,испания,испанский,французский,франция,франция,0.859640,: gram6-nationality-adjective


In [45]:
# res_df1_3 = cos_add_3(dct1)
# save_table(res_df1_3, "3cosadd_3_res")

for key in words_rus:
    print("start {}".format(key))
    res_df1 = cos_add_3(key, dct1)
    save_table(res_df1, "3cosadd_3_res_{}".format(key))
    print("finish {}".format(key))

start : capital-world
finish : capital-world
start : family
finish : family
start : gram1-adjective-to-adverb
finish : gram1-adjective-to-adverb
start : gram2-opposite
finish : gram2-opposite
start : gram3-comparative
finish : gram3-comparative
start : gram6-nationality-adjective
finish : gram6-nationality-adjective


In [46]:
# res_df1["group"].unique()

res_df1 = pd.DataFrame()
for key in groups:
    res_df1 = pd.concat([res_df1, get_table("3cosadd_res_{}".format(key))], ignore_index = True )

In [47]:
res_df1["result"] = res_df1["expected"]  == res_df1["prdicted"] 
count_1 = res_df1.groupby("group").count()
res_sort_1 = res_df1.groupby("group").mean().sort_values(by = "result")
res_sort_1["count"] = count_1["w1"]
res_sort_1 = add_names(res_sort_1)
res_sort_1.index = res_sort_1["type"]
res_sort_1[["cosine", "result", "count"]]

,cosine,result,count
type,,,
Opposite,0.529524,0.000000,2
Comparative,0.667024,0.214286,56
Adjective adverb,0.695183,0.800000,30
Nationality adjective,0.777929,1.000000,11


In [48]:
save_table(res_sort_1, "3cosadd_itog")

In [49]:
# res_df1["group"].unique()

res_df1_3 = pd.DataFrame()
for key in groups:
    res_df1_3 = pd.concat([res_df1_3, get_table("3cosadd_3_res_{}".format(key))], ignore_index = True )

In [50]:
res_df1_3["result"] = res_df1_3["expected"]  == res_df1_3["prdicted"] 
count_1 = res_df1_3.groupby("group").count()
res_3_sort_1 = res_df1_3.groupby("group").mean().sort_values(by = "result")
res_3_sort_1["count"] = count_1["w1"]
res_3_sort_1 = add_names(res_3_sort_1)
res_3_sort_1.index = res_3_sort_1["type"]
res_3_sort_1[["cosine", "result", "count"]]

,cosine,result,count
type,,,
Opposite,0.711460,0.000000,2
Comparative,0.728323,0.410714,56
Adjective adverb,0.710281,0.933333,30
Nationality adjective,0.777929,1.000000,11


In [51]:
save_table(res_3_sort_1, "3cosadd_3_itog")

In [52]:
# cut_b("камбоджа")

###3cosavg

In [53]:
def cos_avg(dct):
    res = pd.DataFrame(columns=["w1","p1","w2","p2","prob", "key","count"])
    for key in words_rus:
        res_df = pd.DataFrame(columns=["w1","w2","w3", "expected", "group"])
        for m_l in words_rus[key]:
            m = []
            for i in m_l:
                
                    m.append(cut_b(i))
                # else:
                #     m.append("none")
        for m in words_rus[key]:
            if m[0] in dct and m[1] in dct and m[2] in dct and m[3] in dct:
                res_df.loc[res_df.shape[0]] = [dct[m[0]], 
                                               dct[m[1]], 
                                               dct[m[3]], 
                                               dct[m[2]],
                                               key]
                # print(m[2], pred)
            
            
        w1,p1 = get_word(res_df["w1"].mean() - res_df["w2"].mean() + res_df["w3"].mean(), dct)
        w2,p2 = get_word(res_df["expected"].mean(), dct)
        prob = 1 - cosine(res_df["w1"].mean() - res_df["w2"].mean() + res_df["w3"].mean(), res_df["expected"].mean())
        res.loc[res.shape[0]] = [w1,p1,w2,p2, prob, key, res_df.shape[0]]
    return res

In [54]:
res_df1_avg = cos_avg(dct1)
res_df1_avg.index = res_df1_avg["key"]
add_names(res_df1_avg)
res_df1_avg.index = res_df1_avg["type"]
res_df1_avg[["w1", "p1", "w2", "p2", "prob", "count"]]

,w1,p1,w2,p2,prob,count
type,,,,,,
Capital-countries,none,-100,none,-100,NaN,0
Family,none,-100,none,-100,NaN,0
Adjective adverb,очевидный,0.90685,очевидный,0.90685,1,30
Opposite,скорее,0.869919,скорее,0.869919,1,2
Comparative,большой,0.860411,большой,0.860411,1,56
Nationality adjective,испания,0.950498,испания,0.949016,0.997948,11


In [55]:
save_table(res_df1_avg, "3cosavg_res")

###3cosmul


In [56]:
def get_word_mul(w1,w2,w3, dct):
    word = "none"
    cs = -100 
    for key in dct:
        e1 = 1 - cosine(dct[w1], dct[key])
        e2 = 1 - cosine(dct[w2], dct[key])
        e3 = 1 - cosine(dct[w3], dct[key])
        if e1 - e2 + e3 > cs and key != w1 and key != w2 and key != w3:
            cs = e1 - e2 + e3
            word = key
    return word, cs

In [57]:
def cos_mul(key, dct):
    res_df = pd.DataFrame(columns=["w1","w2","w3", "expected", "prdicted", "cosine", "group"])
    for m_l in words_rus[key]:
        m = []
        for i in m_l:
            
            m.append(cut_b(i))
            # else:
            #     m.append("none")
    for m in words_rus[key]:
        if m[0] in dct and m[1] in dct and m[2] in dct and m[3] in dct:
            pred, prob = get_word_mul(m[0], m[1], m[3], dct)
            res_df.loc[res_df.shape[0]] = [m[0], m[1], m[3], m[2], pred, prob, key]
                # print(m[2], pred)
    return res_df

In [58]:
for key in words_rus:
    print("start {}".format(key))
    res_df1 = cos_mul(key, dct1)
    save_table(res_df1, "3cosmul_res_{}".format(key))
    print("finish {}".format(key))

start : capital-world
finish : capital-world
start : family
finish : family
start : gram1-adjective-to-adverb
finish : gram1-adjective-to-adverb
start : gram2-opposite
finish : gram2-opposite
start : gram3-comparative
finish : gram3-comparative
start : gram6-nationality-adjective
finish : gram6-nationality-adjective


In [59]:
groups = [": capital-world",
": family",
": gram1-adjective-to-adverb",
": gram2-opposite",
": gram3-comparative",
": gram6-nationality-adjective"]

res_df1_mul = pd.DataFrame()
for key in groups:
    res_df1_mul = pd.concat([res_df1_mul, get_table("3cosmul_res_{}".format(key))], ignore_index = True )

In [60]:
# res_df1_mul = cos_mul(dct1)

# save_table(res_df1_mul, "3cosmul_res")
res_df1_mul["result"] = res_df1_mul["expected"]  == res_df1_mul["prdicted"] 
count_1 = res_df1_mul.groupby("group").count()
res_df_1_mul = res_df1_mul.groupby("group").mean().sort_values(by = "result")
res_df_1_mul["count"] = count_1["w1"]
res_df_1_mul = add_names(res_df_1_mul)
res_df_1_mul.index = res_df_1_mul["type"]
res_df_1_mul[["cosine", "result", "count"]]

,cosine,result,count
type,,,
Opposite,0.795986,0.000000,2
Comparative,0.755446,0.214286,56
Adjective adverb,0.834578,0.800000,30
Nationality adjective,0.911426,1.000000,11


In [61]:
save_table(res_df1_mul, "3cosmul_res")
save_table(res_df_1_mul, "3cosmul_itog")

###mix table

In [ ]:
ddd1 = pd.DataFrame()
def get_r1(r1, metric):
    r1["metric"] = metric
    return r1.pivot(columns = ["metric"])

In [ ]:
ddd1 = get_r1(res_sort_1[["cosine", "result", "count"]], "3CosAdd")
ddd1 = ddd1.merge(get_r1(res_df_1_mul[["cosine", "result", "count"]], "3CosMul"), on = "type")
r1 = res_df1_avg[["prob", "count"]].copy()
r1["cosine"] = r1["prob"]
ddd1 = ddd1.merge(get_r1(r1[["cosine", "count"]], "3CosAvg"), on = "type")
ddd1

In [ ]:
ddd1[["result", "cosine", "count"]]